In [69]:
"""
We've got a mapreduce job that parses through XML files to find every 
unique combination of tags. We've also got some scripts to grab the 
results from S3 and write it to s3 or a csv. 

We need to look through these paths to find the queries that we
care about. For this first bit of scratch code, we'll look for paths
that include `name` but exclude `business`.

The output of this process will be a .csv file 
"""
import pandas as pd

In [70]:
df = pd.read_csv('schema_map.csv', sep='|', names=['form_version_id', 'path'])
df['element_name'] = df.path.str.split('.').apply(lambda x: x[-1])

df.head()

,form_version_id,path,element_name
0,"2013v4.0,IRS990",ReturnData.IRS990ScheduleR.IdDisregardedEntiti...,BusinessNameLine1
1,"2013v4.0,IRS990",ReturnData.IRS990ScheduleH.HealthProfessionsEd...,ActivitiesOrProgramsCnt
2,"2013v4.0,IRS990",ReturnData.IRS990ScheduleA.GiftsGrantsContriRc...,CurrentTaxYearMinus2YearsAmt
3,"2013v4.0,IRS990",ReturnData.IRS990.AllOtherExpensesGrp.Manageme...,ManagementAndGeneralAmt
4,"2013v4.0,IRS990",ReturnData.IRS990.USAddress.ZIPCode,ZIPCode


In [76]:
df['has_name'] = df.element_name.str.lower().str.contains('name')
df['has_business'] = df.element_name.str.lower().str.contains('business')

names = df[(df.has_name) & (~df.has_business)]
# fvi is form_version_id
frequency_of_fvi = names.groupby('path')['element_name'].count()

In [78]:
names.set_index('path', inplace=True, drop=False)
names = names.assign(frequency=frequency_of_fvi)
names.sort_values(by='frequency', ascending=False)

,form_version_id,path,element_name,has_name,has_business,frequency
path,,,,,,
ReturnHeader.Filer.NameControl,"2010v3.6,IRS990EZ",ReturnHeader.Filer.NameControl,NameControl,True,False,54
ReturnHeader.Officer.Name,"2009v1.2,IRS990EZ",ReturnHeader.Officer.Name,Name,True,False,54
ReturnHeader.Filer.NameControl,"2009v1.2,IRS990EZ",ReturnHeader.Filer.NameControl,NameControl,True,False,54
ReturnHeader.Filer.NameControl,"2012v2.3,IRS990EZ",ReturnHeader.Filer.NameControl,NameControl,True,False,54
ReturnHeader.Filer.NameControl,"2009v1.1,IRS990EZ",ReturnHeader.Filer.NameControl,NameControl,True,False,54
ReturnHeader.Officer.Name,"2012v2.1,IRS990",ReturnHeader.Officer.Name,Name,True,False,54
ReturnHeader.Filer.NameControl,"2012v2.1,IRS990PF",ReturnHeader.Filer.NameControl,NameControl,True,False,54
ReturnHeader.Filer.NameControl,"2009v1.4,IRS990PF",ReturnHeader.Filer.NameControl,NameControl,True,False,54
ReturnHeader.Officer.Name,"2010v3.7,IRS990",ReturnHeader.Officer.Name,Name,True,False,54


In [79]:
names.to_csv('paths.csv', index=False)